<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab3/L3_Colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3.3 IO-Link data collection using JSON interface

## 3.1 Python programming for data collection from IO-Link master

The entire schematic of the data communication using JSON interface using REST API for the IO-Link vibraton device from the IO-Link master is illustrated in Figure 15. As the power meter data collection, your computers (laptop and Raspberry Pi) are on the router network. The IO-Link master is also on the same router network with a static IP address. The ethernet cable (RJ45) is used for connection between the router and the IO-Link master. Note that the IO-Link vibration sensor is connected to the **port 1** of the master. In this case using REST API, each of your computer is not a master but client. The IO-Link master is a server. The data communication sequence to read data from the IO-Link master is summarized as follows.

1.  The computer (Raspberry Pi or laptop) requests data using REST API's post method to the IO-Link master 
2.  The IO-Link master read the IO-Link sensor data by the IO-Link protocol
3.  The IO-Link master sends back the received data from the IO-Link sensor to the computer as the JSON format.
4.  The computer receives the JSON data and parse it.

To do this, we will use Python and 'requests' and 'json' packages which are the basic packages in Python. Moreover, refer to details of JSON interface of the IFM IO-Link master [here](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab3/IFM_JSON_Integration_for_IO-Link.pdf?raw=true). The IO-Link vibration sensor (VVB001) manual for interface is also available [here](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab3/IFM_VVB001_IO-Link_Interface_Description.pdf?raw=true). **Please note that this lab manual skips all details to extract data for your convenience. If you perform data collection from other IoT devices, the first thing you have to do is to read the manual and figure out data communication protocols and take appropriate action.**


<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab3_fig15.png?raw=true" width="90%">


*Figure 15 Schematic of communication for IO-Link data collection using JSON interface*


### Task 3.1

Capture the results of checking network connection using ping test from both Raspberry Pi and laptop to the IO-Link master for REST API as Figure 13 ([Lab3.2](https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab3/L3_Colab2.ipynb)) and attach it to the report. 
*  Please note that the IP address of the IO-Link master will be different from the example.




---

Place your screenshot for Task 3.1 here from Raspberry Pi

---





---

Place your screenshot for Task 3.1 here from laptop

---



If ping test works, you are ready to request data. Before we request data to the IO-Link master, let's see what data is available from the sensor. The selected available data is shown in Table 4. These information is also available on the manual [here](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab3/IFM_VVB001_IO-Link_Interface_Description.pdf?raw=true) on page 9. Other than data from Table 4, tens of more data are available. In this lab, we will collect the selected data only. The refresh rate of the sensor is approximately 200 msec in default setting.

**Note that to take an appropreate value with unit, a multiplier in the Range column of Table 4 must be applied after receiving data. For example, after multiplying 0.0001 to v-Rms value, the measured data has unit of m/s.**

*Table 4 Selected process data from VVB001 sensor*

<table width="100%">
<thead>
  <tr>
    <th>Process data input</th>
    <th>Description</th>
    <th>Record Type</th>
    <th>Range</th>
    <th>Unit</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>v-Rms</td>
    <td>Speed effective value</td>
    <td rowspan="5">IntegerT (16 bit)</td>
    <td>(0 to 495)*0.0001</td>
    <td>m/s</td>
  </tr>
  <tr>
    <td>a-Peak</td>
    <td>Acceleration peak value</td>
    <td>(0 to 4903)*0.1</td>
    <td>m/s2</td>
  </tr>
  <tr>
    <td>a_Rms</td>
    <td>Acceleration effective value</td>
    <td>(0 to 4903)*0.1</td>
    <td>m/s2</td>
  </tr>
  <tr>
    <td>Temperature</td>
    <td>Current temperature</td>
    <td>(-300 to 800)*0.1</td>
    <td>°C</td>
  </tr>
  <tr>
    <td>Crest</td>
    <td>Acceleration crest factor</td>
    <td>(10 to 500)*0.1</td>
    <td>-</td>
  </tr>
</tbody>
</table>

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python 3 ([lab3_sample2.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab3/lab3_sample2.py))**

```
import requests
import json
import datetime

# define the IO-Link master IP
URL = "http://192.168.1.x/" # TA will let you know the IP address of the IO-Link master

# define JSON body for post method
# for the details, see the JSON interface manual
# note that the sensor is connected to port 1
BODY = {
        "code":"request",
        "cid":-1,
        "adr":"/iolinkmaster/port[1]/iolinkdevice/iolreadacyclic",
        "data":{"index":40,"subindex":0}    
}

now = datetime.datetime.now()

# this requests data to the IO-Link master
# using POST method of REST API with BODY information
# req object will be response from the IO-Link master
req = requests.post(url = URL, json=BODY)

data_json = req.json() # this is json format data of req

# because the the JSON data itself is in unformatted,
# dumps method of JSON module below helps us to see the data in JSON format
data_json_formatted = json.dumps(data_json, indent=2)

print(now, ': Data structure from the IO-Link master\n',data_json_formatted)

# parsing JSON:
# as you can see the above print,
# 'value' is accesible in 'data' of the JSON data
value = data_json['data']['value']

# the raw measured value looks like 0000FC000002FF000000FF0000F6FF000026FF03
# the length of the value is 40
# this is byte array in 16-bit integer format
print('raw measured value is', value)


# Always, the returned value type from JSON is string
# below is converting the value to v-Rms
# description of v_rms variable is that
# first, select first four elements from value
# second, convert them into 16-bit integer
# third, multiply 0.0001 to take the correct unit (m/s) and data
# finally, rounding up and the data type is float 
v_rms = round(int(value[0:4], 16) * 0.0001)

print('v_Rms =', v_rms ,'m/s')

### indexing information is that
# a_Peak data = value[8:12]
# a_Rms data = value[16:20]
# Temperature data = value[24:28]
# Crest data = value[32:36]

```

---

The sample code 2 (lab3_sample2.py) is given as above. This code is to print out v_Rms. Go through the code and run it. Based on the given code, perform the Tasks below.

### Task 3.2

Model the code block above (lab3_sample1.py) by creating a flow diagram that illustrates the logic, sequence, and processes of the code.

---

Place URL here

---

### Task 3.3

Modify the sample code 2 on both laptop and Raspberry Pi to print out all data (v_Rms, a_Peak, a_Rms, Temperature, and Crest) as Table 4. 

* Print the values with units in Table 4




---

Place your screenshot for Task 3.3 here from Raspberry Pi

---





---

Place your screenshot for Task 3.3 here from laptop

---



### Task 3.4

1.	Perform the same as Task 2.4; Modify sample code 2 to collect IO-Link data for at least 1 minute and save data as a CSV file.

* You can use one computer either laptop or Raspberry Pi.
* Set the sampling period 1 second.
* Data (header) should include time or datetime, v_Rms, a_Peak, a_Rms, Temperature, Crest as Table 5.
* As a practice, plot data and calculate time-domain features.

*Table 5 Example data of output CSV file for Task 3.4*
<table width="100%">
<thead>
  <tr>
    <th>Time [sec]</th>
    <th>v_Rms [m/s]</th>
    <th>a_Peak [m/s2]</th>
    <th>a_Rms [m/s2]</th>
    <th>Temperature [C]</th>
    <th>Crest [-]</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td colspan="6" align="center">...</td>
  </tr>
  <tr>
    <td>10.254</td>
    <td>0.001</td>
    <td>1.24</td>
    <td>0.48</td>
    <td>24.1</td>
    <td>2.58</td>
  </tr>
  <tr>
    <td>10.461</td>
    <td>0.002</td>
    <td>1.34</td>
    <td>0.51</td>
    <td>23.9</td>
    <td>2.63</td>
  </tr>
  <tr>
    <td colspan="6" align="center">...</td>
  </tr>
</tbody>
</table>

In [ ]:
### Place your entire code for Task 3.4 Here
# If you want, you can add more code block below to load csv file and plot it.


## 3.2 Data collection from multiple sensors in a single program

In this lab so far, we collected data from each sensor in a program. In this section, we try to collect data from both sensors at the same time in a single program.

Perform the Tasks below.

### Task 3.5

By using the sample code 1 and 2, program a Python code as instructed below.

* Make infinitely printing out all sensor data at the same time in a 'while' loop.
* Use 'try' and 'except' to halt your program when pressing Ctrl+c.
* Capture your console window or terminal when your program is running and attach it below.
* You can use either laptop or Raspberry Pi to run this code.




---

Place your screenshot for Task 3.5 here.

---



In [ ]:
### place your code for Task 3.5


### Task 3.6

Let's think more about IoT sensors and data collection in the sense of the programmer. The answers are open-ended. Answer the questions below.

#### 1) If you want to collect data from muliple sensors which have different sampling periods, how can you implement your program?



---

Write down your answer to 1) here.

---



#### 2) How can you make your program robust/stable even if suddenly unwanted network connection failure happens?





---

Write down your answer to 2) here.

---



#### 3) What can you do with the electrical power and vibration data? Can they be used for an operational condition or health monitoring?



---

Write down your answer to 3) here.

---



###Task 3.6

Model Unit 4 (prelab + lab) by creating a flow diagram that illustrates the logic, sequence, and processes of the unit.

---

Place URL here

---



## Lab3 Summary and Deliverables

Answer the following questions for your achievements




### Q1. Please summarize Lab3.

---

Write down A1 here.

---


### Q2. What skills did you have to develop to accomplish this project?

---

Wirte down A2 here.

---

### Q3. What aspects of this project were the most beneficial for your learning?

---

Wirte down A3 here.

---

### Q4. What challenges did you encounter in completing the project?

---

Wirte down A4 here.

---

### Q5. How did you overcome the challenges or remedy the problems encountered?

---

Wirte down A5 here.

---

<br></br>

Get back to [Lab Index Page](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/index.ipynb)